In [ ]:
import scanpy as sc
import os,sys,glob
import pandas as pd
import numpy as np
import anndata as ad
import matplotlib.pyplot as plt


import scrublet as scr
import seaborn as sns
#import multiprocessing

In [ ]:
fs = glob.glob(path_dir + '/raw/matrix/*')
fs = list(set([f.split('/')[-1] for f in fs]))
print(len(fs))

# 01. Loading and Merging all data 

In [ ]:
meta = pd.read_csv(path_dir + '/metadata/scRNA_Korean_20240321.csv')

meta['individualID']=meta['individualID'].str.replace("-","")
#meta['Diagnosis']=meta['Final  diagnosis group']
#meta['Clinical_Diagnosis']=meta['Clinical Diagnosis']
meta['Amyloid_positive']=meta['Amyloid positive']
meta['Sex']=meta['Amyloid positive']
meta['Age at death']=meta['age at death']

meta.loc[meta['A']=='A0', 'Alabel'] = 'Low'
meta.loc[meta['A']=='A1', 'Alabel'] = 'Low'
meta.loc[meta['A']=='A2', 'Alabel'] = 'High'
meta.loc[meta['A']=='A3', 'Alabel'] = 'High'

meta.loc[meta['B']=='B0', 'Blabel'] = 'Low'
meta.loc[meta['B']=='B1', 'Blabel'] = 'Low'
meta.loc[meta['B']=='B2', 'Blabel'] = 'High'
meta.loc[meta['B']=='B3', 'Blabel'] = 'High'

meta.loc[meta['C']=='C0', 'Clabel'] = 'Low'
meta.loc[meta['C']=='C1', 'Clabel'] = 'Low'
meta.loc[meta['C']=='C2', 'Clabel'] = 'High'
meta.loc[meta['C']=='C3', 'Clabel'] = 'High'

meta = meta[['individualID','batch',
             'birth','Age at death','Sex',
             'Diagnosis','Clinical_Diagnosis',
             'A','B','C','Alabel','Clabel','Clabel','Amyloid_positive' ]]

meta.head()

In [ ]:
def load_smc(f):
    #print (f)
    adata1 = sc.read_10x_mtx(path_dir + '/raw/matrix/' + f)
    adata1.var_names_make_unique()
    
    # Update meta information
    # Update information    
    obs=adata1.obs.copy()
    obs.reset_index(inplace=True)
    obs.columns=['barcode']
    obs['individualID'] = f

    obs1=pd.merge(obs, meta, on='individualID', how='left')
    obs1.set_index('barcode', inplace=True)

    # Use only intersected barcodes for concat
    intersected_index = obs1.index.intersection(adata1.obs.index)

    # Concat meta information
    obs1 = obs1.loc[intersected_index]
    adata2 = adata1[intersected_index]
    adata2.obs = pd.concat([adata2.obs, obs1], axis=1)
    del adata1
    
    # Update the barcodes
    adata2.name = f
    adata2.obs_names = [f'{adata2.name}_{i}' for i in adata2.obs_names]
    
    
    return adata2

In [ ]:
i=0
results = []
for f in fs:
    i=i+1
    print(i, '/',len(fs),':',f)
    result = load_smc(f)
    results.append(result)

In [ ]:
adata = ad.concat(results, join="outer")

In [ ]:
adata.obs_names_make_unique() 
adata.var_names_make_unique() 

In [ ]:
adata = sc.read_h5ad(path_dir + '/data/scanpy/SMC_Brain.h5ad')

# 02. QC

## 02.01 QC01

In [ ]:
dataset='SMC_Brain'

In [ ]:
adata = sc.read_h5ad(path_dir + '/data/scanpy/SMC_Brain.h5ad')

In [ ]:
adata

In [ ]:
# mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('MT-') 
# ribosomal genes
adata.var['ribo'] = adata.var_names.str.startswith(("RPS","RPL"))
# hemoglobin genes.
adata.var['hb'] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts','total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, groupby='individualID', rotation= 45,
            save='_' + dataset + '_QC_individual_before.pdf'
            )

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts','total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True,
            save='_' + dataset + '_QC_total_before.pdf'
            )

In [ ]:
q1=np.quantile(adata.obs['n_genes_by_counts'], .25)
q3=np.quantile(adata.obs['n_genes_by_counts'], .75)
#print(q1, q3)

upper_gene=q3+3*(q3-q1)
under_gene=q1-3*(q3-q1)
print(under_gene, upper_gene)

q1=np.quantile(adata.obs['total_counts'], .25)
q3=np.quantile(adata.obs['total_counts'], .75)
#print(q1, q3)

upper_total=q3+3*(q3-q1)
under_total=q1-3*(q3-q1)
print(under_total, upper_total)

q1=np.quantile(adata.obs['pct_counts_mt'], .25)
q3=np.quantile(adata.obs['pct_counts_mt'], .75)
#print(q1, q3)

upper_mt=q3+3*(q3-q1)
under_mt=q1-3*(q3-q1)
print(under_mt, upper_mt)


In [ ]:
adata = adata[adata.obs.n_genes_by_counts > under_gene, :]
adata = adata[adata.obs.n_genes_by_counts < upper_gene, :]
adata = adata[adata.obs.total_counts > under_total, :]
adata = adata[adata.obs.total_counts < upper_total, :]
adata = adata[adata.obs.pct_counts_mt > under_mt, :]
adata = adata[adata.obs.pct_counts_mt < upper_mt, :]

In [ ]:
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=10)
adata = adata[adata.obs.pct_counts_mt < 5, :]

In [ ]:
adata

malat1 = adata.var_names.str.startswith('MALAT1')
# we need to redefine the mito_genes since they were first 
# calculated on the full object before removing low expressed genes.
mito_genes = adata.var_names.str.startswith('MT-')
hb_genes = adata.var_names.str.contains('^HB[^(P)]')

remove = np.add(mito_genes, malat1)
remove = np.add(remove, hb_genes)
keep = np.invert(remove)

adata = adata[:,keep]

print(adata.n_obs, adata.n_vars)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts','total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, groupby='individualID',
            save='_' + dataset + '_QC_individual_after.pdf'
            )

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts','total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True,
            save='_' + dataset + '_QC_total_after.pdf'
            )

## 02.03 QC02

In [ ]:
# split per batch into new objects.
batches = adata.obs['individualID'].cat.categories.tolist()
alldata = {}
for batch in batches:
    tmp = adata[adata.obs['individualID'] == batch,]
    print(batch, ":", tmp.shape[0], " cells")
    scrub = scr.Scrublet(tmp.X)
    out = scrub.scrub_doublets(verbose=False, n_prin_comps = 20)
    alldata[batch] = pd.DataFrame({'doublet_score':out[0],'predicted_doublets':out[1]},index = tmp.obs.index)
    print(alldata[batch].predicted_doublets.sum(), " predicted_doublets")
    

In [ ]:
# add predictions to the adata object.
scrub_pred = pd.concat(alldata.values())
adata.obs['doublet_scores'] = scrub_pred['doublet_score'] 
adata.obs['predicted_doublets'] = scrub_pred['predicted_doublets'] 

sum(adata.obs['predicted_doublets'])

In [ ]:
adata = adata[adata.obs['doublet_info'] == 'False',:]
print(adata.shape)

In [ ]:
adata.write(path_dir + '/data/scanpy/SMC_Brain.QC.h5ad')

# 03.Integration(Harmony)

In [ ]:
dataset='SMC_Brain'

In [ ]:
adata = sc.read_h5ad(path_dir + '/data/scanpy/SMC_Brain.QC.h5ad')

In [ ]:
adata

In [ ]:
adata_int=adata.copy()

In [ ]:
# Barcode가 실제 작동하는지 확인하기 위해 UMAP
#adata_int.layers["counts"] = adata_int.X.copy()
sc.pp.normalize_total(adata_int, target_sum=1e4)
sc.pp.log1p(adata_int)

In [ ]:
adata_int

In [ ]:
sc.pp.highly_variable_genes(adata_int, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata_int = adata_int[:, adata_int.var.highly_variable]
#sc.pp.regress_out(adata_int, ['n_counts'])
sc.pp.scale(adata_int, max_value=10)

In [ ]:
sc.tl.pca(adata_int, svd_solver='arpack')

In [ ]:
adata_int.obs['batch'] = adata_int.obs['batch'].astype('category')
adata_int.obs['individualID'] = adata_int.obs['individualID'].astype('category')

In [ ]:
sc.external.pp.harmony_integrate(adata_int, 'individualID')
adata_int.obsm['X_pca'] = adata_int.obsm['X_pca_harmony']

In [ ]:
sc.pp.neighbors(adata_int, n_neighbors=100, n_pcs=20)
sc.tl.leiden(adata_int, resolution=0.8, n_iterations=10)

sc.tl.umap(adata_int)

In [ ]:
col_dict_disease = {
    'Control':'#bfc1c2', # Grey
    'Active control':'#ff7f00',
    'AD':'#e31a1c' # Red
}
sc.pl.umap(adata_int, color=[ 'Diagnosis' ],
           palette=col_dict_disease,
           save='_' + dataset + '_QC_Diagnosis_after.pdf' 
          )

In [ ]:
sc.pl.umap(adata_int, color=[ 'batch' ],
           #palette=col_dict_disease,
           save='_' + dataset + '_QC_batch_after.pdf' 
          )

In [ ]:
sc.pl.umap(adata_int, color=[ 'individualID' ],
           save="_" + dataset + "_QC_individual_after.pdf" 
          )

In [ ]:
col_dict_sex = {
    'Male':'#1f78b4', # Grey
    'Female':'#e31a1c' # Red
}
sc.pl.umap(adata_int, color=[ 'Sex' ],
           palette=col_dict_sex,
           save='_' + dataset + '_QC_Sex_after.pdf' 
          )

In [ ]:
adata.obs['leiden']=adata_int.obs['leiden']

In [ ]:
adata.obsm['X_umap']=adata_int.obsm['X_umap']
adata.obsm['X_pca']=adata_int.obsm['X_pca']
#adata.varm=adata_int.varm

In [ ]:
# Output name
fn = path_dir + '/data/scanpy/SMC_Brain.QC.Harmony.h5ad'

print (fn)

In [ ]:
# Write
adata.write(fn)

# 04 Annotation

In [ ]:
dataset='SMC_Brain'

In [ ]:
infile=path_dir + '/data/scanpy/SMC_Brain.QC.Harmony.h5ad'

In [ ]:
adata = sc.read(infile)

In [ ]:
adata_int=adata.copy()

In [ ]:
adata_int

In [ ]:
sc.pp.normalize_total(adata_int, target_sum=1e4)
sc.pp.log1p(adata_int)

In [ ]:
sc.pp.highly_variable_genes(adata_int, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.neighbors(adata_int, n_neighbors=100, n_pcs=20)

In [ ]:
adata_int

In [ ]:
adata_ref = sc.read_h5ad(path_dir + '/Allen/cortical/Allen_cortex.umap.h5ad')

In [ ]:
adata_ref.obs.class_label.value_counts()

In [ ]:
adata_ref.obs.subclass_label.value_counts()

In [ ]:
obs = adata_ref.obs.copy()
obs = obs [['cell_type_alias_label','subclass_label','class_label']]
obs = obs.drop_duplicates().reset_index(drop=True)
obs.head()

In [ ]:
var_names = adata_ref.var_names.intersection(adata_int.var_names)
var_names

In [ ]:
sc.pp.neighbors(adata_ref, n_neighbors=100, n_pcs=20)

In [ ]:
adata_ref = adata_ref[:, var_names]
adata_int = adata_int[:, var_names]

In [ ]:
sc.tl.ingest(adata_int, adata_ref, obs='cell_type_alias_label')

In [ ]:
adata_int.obs.cell_type_alias_label.value_counts()

In [ ]:
adata_int.obs['cell_type_alias_label'] = adata_int.obs['cell_type_alias_label'].astype('str')

In [ ]:
#subclss
adata_int.obs['subclass_label']=adata_int.obs['cell_type_alias_label']

In [ ]:
for celltype in obs['cell_type_alias_label'] :
    subclass = obs.loc[obs['cell_type_alias_label']==celltype, 'subclass_label'].values[0]
    adata_int.obs.loc[adata_int.obs['cell_type_alias_label']==celltype, 'subclass_label'] = subclass

In [ ]:
col_dict_subclass = {'Astrocyte':'#665c47ff', 
    'Chandelier':'#f641a8ff', 
    'Endothelial':'#8d6c62ff', 
    'L2/3 IT':'#b1ec30ff', 
    'L4 IT':'#00e5e5ff',
    'L5 ET':'#0d5b78ff', 
    'L5 IT':'#50b2adff', 
    'L5/6 NP':'#3e9e64ff', 
    'L6 CT':'#2d8cb8ff', 
    'L6 IT':'#a19922ff',
    'L6 IT Car3':'#5100ffff', 
    'L6b':'#7044aaff', 
    'Lamp5':'#da808cff', 
    'Lamp5 Lhx6':'#935f50ff', 
    'Microglia-PVM':'#94af97ff',
    'OPC':'#374a45ff', 
    'Oligodendrocyte':'#53776cff', 
    'Pax6':'#71238cff', 
    'Pvalb':'#d93137ff', 
    'Sncg':'#df70ffff',
    'Sst':'#ff9900ff', 
    'Sst Chodl':'#b1b10cff', 
    'VLMC':'#697255ff', 
    'Vip':'#a45fbfff'
}

In [ ]:
adata_int.obs['subclass_label'] = adata_int.obs['subclass_label'].astype('str')
#mainclass
adata_int.obs['mainclass_label']=adata_int.obs['subclass_label']
adata_int.obs.loc[adata_int.obs['subclass_label']=='IT', 'mainclass_label'] = 'Ext'
adata_int.obs.loc[adata_int.obs['subclass_label']=='L4 IT', 'mainclass_label'] = 'Ext'
adata_int.obs.loc[adata_int.obs['subclass_label']=='L5 ET', 'mainclass_label'] = 'Ext'
adata_int.obs.loc[adata_int.obs['subclass_label']=='L5/6 NP', 'mainclass_label'] = 'Ext'
adata_int.obs.loc[adata_int.obs['subclass_label']=='L5/6 IT Car3', 'mainclass_label'] = 'Ext'
adata_int.obs.loc[adata_int.obs['subclass_label']=='L6 CT', 'mainclass_label'] = 'Ext'
adata_int.obs.loc[adata_int.obs['subclass_label']=='L6b', 'mainclass_label'] = 'Ext'

adata_int.obs.loc[adata_int.obs['subclass_label']=='PAX6', 'mainclass_label'] = 'IN'
adata_int.obs.loc[adata_int.obs['subclass_label']=='SST', 'mainclass_label'] = 'IN'
adata_int.obs.loc[adata_int.obs['subclass_label']=='PVALB', 'mainclass_label'] = 'IN'
adata_int.obs.loc[adata_int.obs['subclass_label']=='VIP', 'mainclass_label'] = 'IN'
adata_int.obs.loc[adata_int.obs['subclass_label']=='LAMP5', 'mainclass_label'] = 'IN'

adata_int.obs.loc[adata_int.obs['subclass_label']=='Microglia', 'mainclass_label'] = 'MG'
adata_int.obs.loc[adata_int.obs['subclass_label']=='Endothelial', 'mainclass_label'] = 'End'
adata_int.obs.loc[adata_int.obs['subclass_label']=='VLMC', 'mainclass_label'] = 'VLMC'
adata_int.obs.loc[adata_int.obs['subclass_label']=='Oligodendrocyte', 'mainclass_label'] = 'OD'
adata_int.obs.loc[adata_int.obs['subclass_label']=='OPC', 'mainclass_label'] = 'OPC'
adata_int.obs.loc[adata_int.obs['subclass_label']=='Astrocyte', 'mainclass_label'] = 'Ast'
adata_int.obs.loc[adata_int.obs['subclass_label']=='Pericyte', 'mainclass_label'] = 'Per'

In [ ]:
adata

In [ ]:
adata_int

In [ ]:
sc.tl.umap(adata_int)

In [ ]:
col_dict_cell = {
    # Clade 1-1 (cereberum or celebellum)
    'Ext': '#33a02c', # Ext
    'IN': '#e31a1c',        
    'OD': '#ff7f00', # OD
    'Ast': '#1f78b4', # Ast
    'MG': '#6a3d9a', # MG
    'OPC': '#fdbf6f', # OPC
    'End': '#b15928', # End
    'VLMC':'#ffff99',
    'Per':'#ffff99'
    
}

In [ ]:
# Use only intersected barcodes for concat
intersected_index = adata.obs.index.intersection(adata_int.obs.index)

adata = adata[intersected_index]
adata

In [ ]:
sum(adata.obs.index == adata_int.obs.index)

In [ ]:
# Update the cluster names
adata.obs = adata_int.obs

In [ ]:
adata

In [ ]:
sc.pl.umap(
    adata,
    color=["mainclass_label"],
    #legend_loc="on data",
    palette=col_dict_cell,
    #legend_fontoutline=1,
    save="_" + dataset + "_mainclass_label.pdf"
)

In [ ]:
sc.pl.umap(adata, 
           color=['subclass_label' ],
           #palette=col_dict_subclass,
           save="_"+dataset+"_subclass_label.pdf" 
          )

In [ ]:
infile=path_dir + '/data/scanpy/SMC_Brain.QC.Harmony.anno.h5ad'
adata.write(infile)

# 05. Save Seurat raw 

In [ ]:
import scanpy as sc
import pandas as pd
from scipy import io
import os

In [ ]:
dataset='SMC_Brain'

In [ ]:
infile=path_dir + '/data/scanpy/SMC_Brain.QC.Harmony.anno.h5ad'

In [ ]:
adata = sc.read(infile)

In [ ]:
adata.obs.mainclass_label.value_counts()

In [ ]:
class_list=adata.obs.mainclass_label.unique()
class_list

In [ ]:
with open(path_result + '/barcodes.tsv', 'w') as f:
        for item in adata.obs_names:
            f.write(item + '\n')
with open(path_result + '/features.tsv', 'w') as f:
        for item in ['\t'.join([x,x,'Gene Expression']) for x in adata.var_names]:
            f.write(item + '\n')
io.mmwrite(path_result + '/matrix', adata.X.T)
adata.obs.to_csv( path_result + '/metadata.csv')

In [ ]:
for cls in class_list :
    print(cls)
    try:
        os.mkdir(path_result + cls)
    except OSError as error:
        print(error)
    adata1=adata[adata.obs.mainclass_label==cls]
    with open(path_result + cls + '/barcodes.tsv', 'w') as f:
        for item in adata1.obs_names:
            f.write(item + '\n')
    with open(path_result + cls + '/features.tsv', 'w') as f:
        for item in ['\t'.join([x,x,'Gene Expression']) for x in adata1.var_names]:
            f.write(item + '\n')
    io.mmwrite(path_result + cls + '/matrix', adata1.X.T)
    adata1.obs.to_csv( path_result + cls + '/metadata.csv')

# END